In [2]:
%pip install adapters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 118.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.1
    Uninstalling transformers-4.53.1:
      Successfully uninstalled transformers-4.53.1
ERROR: Operation cancelled by user


In [5]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 80.1 MB/s eta 0:00:00


In [1]:
import torch
from transformers import CLIPTokenizer, CLIPTextModel, BertTokenizer, CLIPModel
# from transformers.adapters import BertAdapterModel
from torch.utils.data import DataLoader, Dataset
from torch.nn import MSELoss
# import transformers.adapters.composition as ac

In [2]:
# -----------------------
# 1. Setup: Teacher (CLIP) and Student (BERT + Adapter)
# -----------------------

# Teacher: CLIP text encoder
clip_tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
clip_model.eval()
for param in clip_model.parameters():
    param.requires_grad = False

# Student: BERT with adapter
student_tokenizer = BertTokenizer.from_pretrained("HooshvareLab/bert-fa-base-uncased")
# Load the base model first
from transformers import AutoModel
from peft import LoraConfig, get_peft_model

student_model = AutoModel.from_pretrained("HooshvareLab/bert-fa-base-uncased")

# Add adapter for distillation using a PeftConfig
adapter_name = "clip_distill_adapter"
config = LoraConfig(
    r=8, # rank
    lora_alpha=16,
    target_modules=["query", "value"], # commonly used for text models
    lora_dropout=0.1,
    bias="none",
    task_type="FEATURE_EXTRACTION" # or "SEQ_CLS", etc. depending on your task
)
student_model = get_peft_model(student_model, config, adapter_name=adapter_name)

# student_model.train_adapter(adapter_name) # This is not needed with peft

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

In [3]:
!curl https://raw.githubusercontent.com/merlin6990/Video-Retrieval-System/refs/heads/main/Data/output.json -o data.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4724k  100 4724k    0     0  7455k      0 --:--:-- --:--:-- --:--:-- 7451k


In [4]:
# Example usage
import json

# Replace 'your_json_string' with your actual JSON data as a string
with open("data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [5]:
def extract_sentences_and_translations(data):
    results = []

    def recursive_extract(obj):
        if isinstance(obj, dict):
            # If the object has an "annotations" key, process its list
            if "annotations" in obj and isinstance(obj["annotations"], list):
                for annotation in obj["annotations"]:
                    if isinstance(annotation, dict):
                        sentence = annotation.get("sentence")
                        translation = annotation.get("translation")
                        if sentence is not None and translation is not None:
                            results.append((sentence, translation))
            # Continue searching through the rest of the dict
            for value in obj.values():
                recursive_extract(value)
        elif isinstance(obj, list):
            for item in obj:
                recursive_extract(item)

    recursive_extract(data)
    return results



In [6]:
pairs = []

for item in data.values():
    pairs.extend(extract_sentences_and_translations(item))

In [7]:
pairs[1000]

('cover the bowl with saran wrap', 'کاسه را با نوار پوشانده')

In [8]:
# -----------------------
# 2. Sample Dataset
# -----------------------

class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = pairs

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        english = self.texts[idx][0]
        persian = self.texts[idx][1]
        return {
            "english": english,
            "persian": persian
        }


dataset = TextDataset(pairs)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [9]:
loader.__iter__().__next__()

{'english': ['add red pepper flakes to the pot',
  'add bean sprouts and chives',
  'put another piece of white bread on the top of it',
  'spread some butter on the nonstick skillet',
  'place sauce and chicken on the pasta',
  'add sugar to the wok and mix',
  'pour the beef stock into the pan and season with some black pepper salt and bisto powder',
  'bake the mussels at 250f for 7-10 minutes',
  'place the pan into the oven',
  'cook the rice with water seasoning oil and yellow coloring',
  'place the eggs on top of the salmon',
  'add pasta to pot',
  'wash the clams under running water with hand and place them in a pot',
  'fry some paste in a pan with oil',
  'add clam juice 2 potatoes and bacon',
  'crush the garlic mash with salt and chop'],
 'persian': ['دانه های فلفل قرمز را به دیگ اضافه کنید',
  'دانه های لوبیا و تره فرنگی را اضافه کنید',
  'یک تکه نان سفید دیگر روی آن بگذار',
  'مقداری کره روی ماهیتابه بدون دسته پهن کنید',
  'سس و مرغ را روی پاستا بگذار',
  'شکر را به دیگ

In [10]:
# -----------------------
# 3. Training Loop
# -----------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clip_model.to(device)
student_model.to(device)

optimizer = torch.optim.Adam(student_model.parameters(), lr=1e-4)
loss_fn = MSELoss()

for epoch in range(3):
    x = 0
    for batch in loader:
        # Teacher (CLIP): get text features from pooler_output (dim=768)
        clip_inputs = clip_tokenizer(batch['english'], padding=True, return_tensors="pt").to(device)
        with torch.no_grad():
            # Access text features from the pooler_output of the CLIPTextModel output
            teacher_embeds = clip_model.get_text_features(**clip_inputs)

        # Student (BERT): CLS token output (dim=768)
        student_inputs = student_tokenizer(batch['persian'], padding=True, return_tensors="pt").to(device)
        student_output = student_model(**student_inputs).last_hidden_state[:, 0, :]  # [CLS]

        # Loss and optimization
        loss = loss_fn(student_output, teacher_embeds)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if x % 1000 == 0:
            print(f"Epoch {epoch + 1} | Batch {x + 1} | Loss: {loss.item():.4f}")
        x += 16

Epoch 1 | Batch 1 | Loss: 1.1584
Epoch 1 | Batch 2001 | Loss: 0.4478
Epoch 1 | Batch 4001 | Loss: 0.3785
Epoch 1 | Batch 6001 | Loss: 0.3714
Epoch 1 | Batch 8001 | Loss: 0.3718
Epoch 1 | Batch 10001 | Loss: 0.3753
Epoch 1 | Batch 12001 | Loss: 0.3561
Epoch 2 | Batch 1 | Loss: 0.3172
Epoch 2 | Batch 2001 | Loss: 0.3476
Epoch 2 | Batch 4001 | Loss: 0.3538
Epoch 2 | Batch 6001 | Loss: 0.3649
Epoch 2 | Batch 8001 | Loss: 0.3514
Epoch 2 | Batch 10001 | Loss: 0.3095
Epoch 2 | Batch 12001 | Loss: 0.3101
Epoch 3 | Batch 1 | Loss: 0.2910
Epoch 3 | Batch 2001 | Loss: 0.3344
Epoch 3 | Batch 4001 | Loss: 0.3303
Epoch 3 | Batch 6001 | Loss: 0.3512
Epoch 3 | Batch 8001 | Loss: 0.3507
Epoch 3 | Batch 10001 | Loss: 0.3557
Epoch 3 | Batch 12001 | Loss: 0.3426


In [11]:
# -----------------------
# 4. Save Adapter
# -----------------------

student_model.save_pretrained("clip_distill_adapter/")
print("✅ Adapter saved.")

✅ Adapter saved.


In [31]:
import torch
from transformers import CLIPProcessor, CLIPVisionModel, BertTokenizer, AutoModel
from peft import PeftModelForFeatureExtraction # Assuming FEATURE_EXTRACTION task type
from torch.nn.functional import cosine_similarity
from PIL import Image
import requests

processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


student_model.eval()

# --- Encode an example image ---
# You can replace this with your own image loading logic
image_url = "https://i.ytimg.com/vi/YRLiV17UCZ8/hq720.jpg?sqp=-oaymwEhCK4FEIIDSFryq4qpAxMIARUAAAAAGAElAADIQj0AgKJD&rs=AOn4CLBtjHlpQhT51hO1Ios995ojuJgGxQ" # Example image URL
image = Image.open(requests.get(image_url, stream=True).raw)

# Process the image and get embeddings
with torch.no_grad():
    image_inputs = processor(images=image, return_tensors="pt").to(device)
    # CLIPVisionModel typically outputs a pooled output for the image embedding
    image_embedding = clip_model.get_image_features(**image_inputs)

print("Image embedding shape:", image_embedding.shape)

# --- Encode an example text ---
# Example text (in Persian)
text_to_encode = ["دانه های لوبیا و تره فرنگی را اضافه کن",
                  "سرخ کردن سبزیجات در ماهیتابه با قاشق چوبی",
                  "تخم مرغ را در یک ظرف بشکنید.",
                  "پنیر را روی نان بگذار",
                  "ماهی‌تابه را در اجاق بگذار.",
                  ]

# Tokenize the input text
text_inputs = student_tokenizer(text_to_encode, return_tensors="pt", padding=True, truncation=True)

# Move inputs to the same device as the model
text_inputs = {key: value.to(device) for key, value in text_inputs.items()}

# Get the text embedding
with torch.no_grad():
    # Assuming the task type is FEATURE_EXTRACTION and CLS token is used for representation
    text_embedding = student_model(**text_inputs).last_hidden_state[:, 0, :]

print("Text embedding shape:", text_embedding.shape)

# --- Calculate Cosine Similarity ---
# Ensure embeddings are normalized if necessary (CLIP embeddings are usually normalized)
# You might want to normalize the student text embeddings as well for better comparison
image_embedding_normalized = image_embedding / image_embedding.norm(p=2, dim=-1, keepdim=True)
text_embedding_normalized = text_embedding / text_embedding.norm(p=2, dim=-1, keepdim=True)


cosine_sim = cosine_similarity(image_embedding_normalized, text_embedding_normalized, dim=1)

print("\nCosine Similarity:", cosine_sim)

Image embedding shape: torch.Size([1, 768])
Text embedding shape: torch.Size([5, 768])

Cosine Similarity: tensor([0.1782, 0.1974, 0.2285, 0.2302, 0.2391], device='cuda:0')


In [32]:
import torch.nn.functional as F

# Apply softmax to convert cosine similarities to probabilities
probabilities = F.softmax(cosine_sim, dim=0)

# Print the probabilities
for i, (text, prob) in enumerate(zip(text_to_encode, probabilities)):
    print(f"{i+1}. \"{text}\" - Probability: {prob.item():.4f}")


1. "دانه های لوبیا و تره فرنگی را اضافه کن" - Probability: 0.1928
2. "سرخ کردن سبزیجات در ماهیتابه با قاشق چوبی" - Probability: 0.1965
3. "تخم مرغ را در یک ظرف بشکنید." - Probability: 0.2027
4. "پنیر را روی نان بگذار" - Probability: 0.2031
5. "ماهی‌تابه را در اجاق بگذار." - Probability: 0.2049


In [ ]:
from peft import PeftModel

student_.push_to_hub("your-username/my-adapter")
